## 대용량 데이터(9G) 용량 줄이기  
1. csv 파일 로드하기 : pd.read_csv('data/2019-Oct.csv')
2. column 별 데이터 값 확인하여 필요 없는 col 없애기 : df.drop(columns=[])
3. 각 열 데이터 형식 확인하기
4. 각 열 downcast 하기 : pd.to_numeric(df[col], downcast='')
 => for 문을 통해 자동으로 변환하기
5. parquet.gzip으로 파일 저장하기 : df.to_parquet(file_name, index=False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/2019-Nov.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.5+ GB


In [3]:
df_down = df.copy()
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.5+ GB


In [4]:
df_down = df_down.drop(columns=['product_id', 'category_id', 
                                'category_code', 'brand', 'user_session'])
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   event_time  object 
 1   event_type  object 
 2   price       float64
 3   user_id     int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 2.0+ GB


In [5]:
df_down.isnull().sum()

event_time    0
event_type    0
price         0
user_id       0
dtype: int64

In [7]:
for col in df_down.columns:
    
    if df_down[col].dtypes.name.startswith('int'):
        if df_down[col].min() < 0:
            df_down[col] = pd.to_numeric(df_down[col], downcast='integer')
        else:
            df_down[col] = pd.to_numeric(df_down[col], downcast='unsigned')
    
    elif df_down[col].dtypes.name.startswith('float'):
        df_down[col] = pd.to_numeric(df_down[col], downcast='float')
        
    elif df_down[col].dtypes.name.startswith('object') & (df_down[col].nunique() < 3500):
        df_down[col] = df_down[col].astype('category') 

In [8]:
df_down.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 4 columns):
 #   Column      Dtype   
---  ------      -----   
 0   event_time  object  
 1   event_type  category
 2   price       float32 
 3   user_id     uint32  
dtypes: category(1), float32(1), object(1), uint32(1)
memory usage: 1.1+ GB


In [9]:
df_down['event_time'] = pd.to_datetime(df_down['event_time'])

# parquet 포멧으로 파일 저장하기
- donwcast 후 csv 파일과 parquet 저장한 파일 용량 비교하기

In [10]:
import os

In [11]:
def convert_bytes(num):
    """
    1024 보다 크면 숫자를 나누고 아니면 숫자와 단위를 표시하도록
    for문을 돌면서 값을 1024로 나누고 
    값이 1024 보다 작다면 단위와 함께 num 을 반횐
    """
    for file_size in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024:
            return f'{num:.2f}{file_size}'
            break
        num /= 1024

In [12]:
def file_size(file_path):
    """
    파일이 있다면 convert_bytes 함수를 통해 크기를 구함
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [13]:
def compare_csv_parquet(df):
    """
    데이터프레임을 csv 와 parquet형식으로 저장하하고 각 파일 사이즈를 dict 형태로 반환
    """
    file_path_parquet = 'data/2019-Nov_downcast.parquet.gzip'
    file_path_csv = 'data/2019-Nov.csv'
    
    df_down.to_parquet(file_path_parquet, compression='gzip', index=False)
  
    return ({'parquet': file_size(file_path_parquet), 'csv': file_size(file_path_csv)})

In [14]:
compare_csv_parquet(df_down)

{'parquet': '315.24MB', 'csv': '8.39GB'}